In [2]:
import json
from typing import Type

from dotenv import load_dotenv
from langchain_community.agent_toolkits.github.toolkit import GitHubToolkit
from langchain_community.utilities.github import GitHubAPIWrapper
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from pydantic import BaseModel

load_dotenv()

# Instantiate GitHub Toolkit
toolkit = GitHubToolkit.from_github_api_wrapper(GitHubAPIWrapper())
tools = [setattr(tool, "name", tool.mode) or tool for tool in toolkit.get_tools()]

llm = ChatOpenAI(model="gpt-4o", temperature=0)

def ai(query: str, response_format: Type[BaseModel]):
    agent_executor = create_react_agent(llm, tools, response_format=response_format)
    events = agent_executor.stream(
        {"messages": [("user", query)]},
        stream_mode="values",
    )
    event = {}
    for event in events:
        event["messages"][-1].pretty_print()

    structured_response = event.get("structured_response")
    data = {"data": [structured_response.model_dump()]}
    print(data)
    return json.dumps(data)


In [3]:
from dotenv import load_dotenv
from flask import Flask
from flask_cors import CORS
from pydantic import BaseModel

# Load environment variables from .env
load_dotenv()

app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "http://localhost:3000"}})


class Author(BaseModel):
    """Respond to the user in this format."""
    name: str
    issue_count: int
    comment_count: int


@app.route('/data', methods=['GET'])
def get_data():
    query = ("For the most prolific author of github issues, give me the number of issues they have created. "
             "Given the author's issues, I want to know the number of comments that same author has made." )

    # We are trying to get rid of engineering as much as possible, but Author and tools implementation is still in the way
    return ai(query, response_format=Author)
    

## Goals
- ~~Write natural language only~~
- ~~Unstructured data~~
- Implicit algorithms
- ~~Dynamic tools~~
- ~~Human in the loop built-in~~
- ~~Consistent output~~
- ~~Iterative development~~

In [4]:
app.run(debug=False, port=5001, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit


================================ Human Message =================================

For the most prolific author of github issues, give me the number of issues they have created. Given the author's issues, I want to know the number of comments that same author has made.
================================== Ai Message ==================================
Tool Calls:
  get_issues (call_VvT4ynAuuM6HS6Wp0TWA2ibo)
 Call ID: call_VvT4ynAuuM6HS6Wp0TWA2ibo
  Args:
    no_input:
================================= Tool Message =================================
Name: get_issues

Found 2 issues:
[{'title': 'This is my second issue', 'number': 2, 'opened_by': 'yasonk'}, {'title': 'First issue here you go', 'number': 1, 'opened_by': 'yasonk'}]
================================== Ai Message ==================================
Tool Calls:
  get_issue (call_zTNlLpRi7Zu6WmGCgG9LuWKc)
 Call ID: call_zTNlLpRi7Zu6WmGCgG9LuWKc
  Args:
    issue_number: 2
  get_issue (call_Vcdfvsve4RPsEBCzaAlFqnhk)
 Call ID: call_Vcd

127.0.0.1 - - [26/Feb/2025 19:06:58] "GET /data HTTP/1.1" 200 -


================================== Ai Message ==================================

The most prolific author of GitHub issues is "yasonk," who has created 2 issues. This author has also made a total of 3 comments on their issues.
{'data': [{'name': 'yasonk', 'issue_count': 2, 'comment_count': 3}]}
================================ Human Message =================================

For the most prolific author of github issues, give me the number of issues they have created. Given the author's issues, I want to know the number of comments that same author has made.
================================== Ai Message ==================================
Tool Calls:
  get_issues (call_ZnrztDxohc9Q71Jihb6uxx6A)
 Call ID: call_ZnrztDxohc9Q71Jihb6uxx6A
  Args:
    no_input:
================================= Tool Message =================================
Name: get_issues

Found 2 issues:
[{'title': 'This is my second issue', 'number': 2, 'opened_by': 'yasonk'}, {'title': 'First issue here you go', 'number

127.0.0.1 - - [26/Feb/2025 19:08:07] "GET /data HTTP/1.1" 200 -


================================== Ai Message ==================================

The most prolific author of GitHub issues is "yasonk," who has created 2 issues. Additionally, "yasonk" has made a total of 3 comments on these issues.
{'data': [{'name': 'yasonk', 'issue_count': 2, 'comment_count': 3}]}
